### Tokenizer Template 
- instruction-tuning 템플릿으로 변환했을 때 masking 여부 확인

In [1]:
import numpy as np
from colorama import Fore
from icecream import ic
from transformers import AutoTokenizer


MODEL_DIR = "/home/hyunsoo.lee/disk1/llm-models/llama-160m"
MODEL_DIR = "/home/hyunsoo.lee/disk1/llm-models/CodeLlama-13b-hf"
MODEL_DIR = "/home/hyunsoo.lee/disk1/llm-models/phi-2"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    add_special_tokens=False,
)
tokenizer.padding_side = "right"
print(tokenizer.eos_token)
tokenizer

</s>


LlamaTokenizerFast(name_or_path='JackFram/llama-160m', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [2]:
def merge(instruction_template, instruction, response_template, response):
    return instruction_template + instruction.strip() + response_template + response


def is_included(s, target):
    for i in range(len(s)):
        cnt = 0
        for si, ti in zip(s[i:], target):
            if si != ti:
                break
            cnt += 1
        if cnt == len(target):
            return True
    return False


def check(instruction_template, instruction, response_template, response):
    input_text = merge(instruction_template, instruction, response_template, response)
    print(Fore.WHITE + input_text)

    input_text_tokenized = tokenizer.encode(input_text)
    merged = [
        tokenizer.encode(x, add_special_tokens=False)
        for x in [instruction_template, instruction, response_template, response]
    ]
    print(Fore.YELLOW + " + ".join(tokenizer.decode(x) for x in input_text_tokenized))

    instruction_template_tokenized = tokenizer.encode(
        instruction_template, add_special_tokens=False
    )
    response_template_tokenized = tokenizer.encode(
        response_template, add_special_tokens=False
    )[1:]
    # print(Fore.WHITE)
    # print(instruction_template_tokenized)
    # print(" + ".join(tokenizer.decode(x) for x in instruction_template_tokenized))
    # print(response_template_tokenized)
    # print(" + ".join(tokenizer.decode(x) for x in response_template_tokenized))

    print(Fore.GREEN)
    i_included = is_included(input_text_tokenized, instruction_template_tokenized)
    o_included = is_included(input_text_tokenized, response_template_tokenized)
    print(f"instruction_template_included: {i_included}")
    print(f"response_template_included: {o_included}")

In [3]:
instruction = "your name?"
response = "guy"

templates = [
    ("### Question:\n", "\n\n### Answer:\n"),
    ("Question:\n", "\nAnswer:\n"),
    ("[INST]\n", "\n[/INST]\n"),
    ("[INST] ", " [/INST]"),
]

for instruction_template, response_template in templates:
    check(instruction_template, instruction, response_template, response)
    print()

### Question:
your name?

### Answer:
guy
<s> + ### + Question + : + 
 + your + name + ? + 
 + 
 + ## + # + Answer + : + 
 + gu + y

instruction_template_included: True
response_template_included: True

Question:
your name?
Answer:
guy
<s> + Question + : + 
 + your + name + ? + 
 + Answer + : + 
 + gu + y

instruction_template_included: True
response_template_included: True

[INST]
your name?
[/INST]
guy
<s> + [ + INST + ] + 
 + your + name + ? + 
 + [ + / + INST + ] + 
 + gu + y

instruction_template_included: True
response_template_included: True

[INST] your name? [/INST]guy
<s> + [ + INST + ] + your + name + ? + [ + / + INST + ] + gu + y

instruction_template_included: False
response_template_included: True

